In [21]:
import pandas as pd
import re
import numpy as np

In [22]:
df = pd.read_csv('data/suumo_minato.csv', sep=',', index_col=0)
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1253 entries, 0 to 1252
Data columns (total 14 columns):
新着      0 non-null float64
物件名     1253 non-null object
住所      1253 non-null object
立地1     1253 non-null object
立地2     1250 non-null object
立地3     1240 non-null object
築年数     1253 non-null object
階数      1253 non-null object
物件階     1253 non-null object
家賃      1253 non-null object
管理費     1253 non-null object
敷金礼金    1253 non-null object
間取り     1253 non-null object
面積      1253 non-null object
dtypes: float64(1), object(13)
memory usage: 146.8+ KB


In [38]:
df.head()

,新着,物件名,住所,立地1,立地2,立地3,築年数,階数,物件階,家賃,管理費,敷金礼金,間取り,面積,都県,市区,家賃合計,家賃単価（円/平米）
0,NaN,グランクルール武蔵小山,東京都品川区小山３,東急目黒線/武蔵小山駅 歩5分,東急池上線/戸越銀座駅 歩13分,東急目黒線/不動前駅 歩13分,3,4階建,3階,87000,12000.0,-/-,1K,21.50,東京,品川,99000.0,4604.651163
1,NaN,グランクルール武蔵小山,東京都品川区小山３,東急目黒線/武蔵小山駅 歩5分,東急池上線/戸越銀座駅 歩13分,東急目黒線/不動前駅 歩13分,3,4階建,3階,97000,12000.0,-/-,ワンルーム,21.50,東京,品川,109000.0,5069.767442
2,NaN,グランクルール武蔵小山,東京都品川区小山３,東急目黒線/武蔵小山駅 歩5分,東急池上線/戸越銀座駅 歩13分,東急目黒線/不動前駅 歩13分,3,4階建,3階,97000,12000.0,-/-,ワンルーム,21.50,東京,品川,109000.0,5069.767442
3,NaN,カスタリア大井町,東京都品川区大井４,ＪＲ京浜東北線/大井町駅 歩4分,京急本線/鮫洲駅 歩14分,ＪＲ横須賀線/西大井駅 歩15分,8,14階建,14階,97000,8000.0,9.7万円/9.7万円,1K,21.75,東京,品川,105000.0,4827.586207
4,NaN,カスタリア大井町,東京都品川区大井４,ＪＲ京浜東北線/大井町駅 歩4分,京急本線/鮫洲駅 歩14分,ＪＲ横須賀線/西大井駅 歩15分,8,14階建,11階,98000,8000.0,19.6万円/9.8万円,1K,21.75,東京,品川,106000.0,4873.563218


In [24]:
# 文字列を複数文字で split
df['都県'] = df['住所'].apply(lambda x: re.split('[都県]', x)[0]) 
df['市区'] = df['住所'].apply(lambda x: re.split('[都県市区]', x)[1]) 

In [25]:
df['築年数'].unique()

array(['築3年', '築8年', '築1年', '築7年', '新築', '築2年', '築10年', '築6年', '築4年',
       '築5年', '築9年', '築0年'], dtype=object)

In [26]:
df['築年数'] = df['築年数'].str.strip('築年')
df['築年数'] = df['築年数'].str.replace('新', '0')
df['築年数'] = pd.to_numeric(df['築年数'])

In [27]:
sum(df['家賃'].apply(lambda x: '万円' not in x))

0

In [28]:
df['家賃'] = df['家賃'].str.replace('万円', '').apply(lambda x: int(float(x) * 10000))

In [29]:
df['管理費'] = df['管理費'].str.replace('-', '')
df['管理費'] = df['管理費'].str.replace('円', '')
df['管理費'] = pd.to_numeric(df['管理費'], downcast='integer')
df['管理費'] = df['管理費'].apply(lambda x: 0 if np.isnan(x) else x)

In [31]:
sum(df['面積'].apply(lambda x: 'm' not in x))

0

In [32]:
df['面積'] = df['面積'].str.strip('m')
df['面積'] = pd.to_numeric(df['面積'])

In [37]:
df['家賃合計'] = df['家賃'] + df['管理費']
df['家賃単価（円/平米）'] = df['家賃合計'] / df['面積']

In [39]:
df.to_csv('data/suumo_tokyo_mod.csv')